<img src="https://www.mlflow.org/docs/latest/_static/MLflow-logo-final-black.png" width="200">
#### MLflow Lab Part 3: Model Registry Demo
         
The goal of this lab to demonstrate how easy it is to register models with different `flavors` and to transition them trough different stages (using MLflow UI and MLflow API for that)

**What we will do (core principles):**
1. Train and track a XGBoost model
2. Register the best iteration of that model and stage it in `production`
3. Load and evaluate the current `production` model
4. Create a second model version using TensorFlow
5. Transition this model to `staging`
6. Compare both `staging` and `production` models
7. Transition the model from `staging` to `production`

#### Classification Problem 

This is a very simple classification problem where we will be using the Fashion MNIST dataset from Keras.

<img src="https://timesofdatascience.com/wp-content/uploads/2019/02/fashion-846x515.jpg"
         alt="Fashion MNIST dataset " width="400">
         
**The Fasion MNIST dataset includes:**

* 60,000 training examples
* 10,000 testing examples
* 10 classes 
* 28x28 grayscale/single channel images

**The ten fashin labels include:**

1. T-shirt/top
2. Trouser/pants
3. Pullover shirt
4. Dress
5. Coat
6. Sandal
7. Shirt
8. Sneaker
9. Bag
10. Ankle boot

In [0]:
%run ./classes_init

### Get Training and Validation Data

In [0]:
(x_train, y_train), (val_x, val_y) = Utils.load_data()

## Part 1: Use XGBoost to create and register the first model

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png"  width="200">

### 1.1: Train the model using XGBoost and track it with MLflow

Iterate over three different set of tunning parameters and track all its results:
 * max_depth
 * eta
 * num_round

In [0]:
for (max_depth, eta, num_round) in [(10, 0.1, 2), (5, 0.05, 15), (15, 0.1, 20)]:
  XGBoost_obj = XGBoostModel(x_train, val_x, y_train, val_y,max_depth=max_depth, eta=eta, num_round=num_round)
  run_id = XGBoost_obj.mlflow_run()
  print(f"Finished running XGBOOST training with run_id={run_id}")

# Register the best XGBoost model with the MLflow Model Registry

Now that the XGBoost model has been trained and tracked with MLflow, the next step is to register it with the MLflow Model Registry. You can register and manage models using the MLflow UI.

### Create a new Registered Model

1. First, navigate to the MLflow Runs Sidebar by clicking the `Runs` icon in the Databricks Notebook UI.


2. Next, locate the MLflow Run corresponding to the Keras model training session, and open it in the MLflow Run UI by clicking the `View Run Detail` icon.

3. In the MLflow UI, scroll down to the `Artifacts` section and click on the directory named `model`. Click on the `Register Model` button that appears.

4. Then, select `Create New Model` from the drop-down menu, and input the following model name: `Fashion_MNISTmodel`. Finally, click `Register`. This registers a new model called `Fashion_MNISTmodel` and creates a new model version: `Version 1`.

After a few moments, the MLflow UI displays a link to the new registered model. Follow this link to open the new model version in the MLflow Model Registry UI.

### Explore the Model Registry UI Workflow

1. The Model Version page in the MLflow Model Registry UI provides information about `Version 1` of the registered forecasting model, including its author, creation time, and its current stage.

2. The Model Version page also provides a `Source Run` link, which opens the MLflow Run that was used to create the model in the MLflow Run UI. From the MLflow Run UI, you can access the `Source Notebook Link` to view a snapshot of the Databricks Notebook that was used to train the model.

To navigate back to the MLflow Model Registry, click the `Models` icon in the Databricks Workspace Sidebar. The resulting MLflow Model Registry home page displays a list of all the registered models in your Databricks Workspace, including their versions and stages.

Select the `Fashion_MNISTmodel` link to open the Registered Model page, which displays all of the versions of the forecasting model.

### Add model descriptions

Add a high-level description to the registered power forecasting model by clicking the `Edit Description` icon, entering the following description, and clicking `Save`:

```
This model forecasts the power output of a wind farm based on weather data. The weather data consists of three features: wind speed, wind direction, and air temperature.
```

Next, click the `Version 1` link from the Registered Model page to navigate back to the Model Version page. Then, add a model version description with information about the model architecture and machine learning framework; click the `Edit Description` icon, enter the following description, and click `Save`:
```
This model version was built using Keras. It is a feed-forward neural network with one hidden layer.
```

### Perform a model stage transition

Click the `Stage` button to display the list of available model stages and your available stage transition options. Select `Transition to -> Production` and press `OK` in the stage transition confirmation window to transition the model to `Production`.

In [0]:
model_name = "Fashion_MNISTmodel"

## Load versions of the registered model

You can load a model by specifying its name (e.g., `Fashion_MNISTmodel`) and version number (e.g., `1`). The following cell uses the `mlflow.pyfunc.load_model()` API to load `Version 1` of the registered power forecasting model as a generic Python function.

In [0]:
import mlflow.pyfunc

model_version_uri = "models:/{model_name}/1".format(model_name=model_name)

print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_version_uri))
model_version_1 = mlflow.pyfunc.load_model(model_version_uri)


You can also load a specific model stage. The following cell loads the `Production` stage of the power forecasting model.

In [0]:
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_production_uri))
model_production = mlflow.pyfunc.load_model(model_production_uri)

## Generate predictions with the production model

In this section, the production model is used to classify input images. The `PlotUtils.predticClassification()` class loads and calls the model from the specified URI and returns an array containing the predicted image labels. This array is then used by `PlotUtils.confusionMatrix()` to generate a Confusion Matrix.

In [0]:
prediction= PlotUtils.predticClassification (model_version_uri, val_x.reshape(val_x.shape[0],-1))
                                             
PlotUtils.confusionMatrix(val_y, prediction, classes= list(range(10)))

## Part 2: Use TensorFlow to create the second model. 

<img src="https://deepadvisors.com/static/tensorflow-logo-min-c79ea9da455c51eb37dcb4d55e5c239c.png"  width="200">

This model, in real development environment, could be developed by another developer and registered in the model registry. 

Use MLflow to compare and then elect the best model between XGBoost and TensorFlow for production.

### 2.1 Train the TensorFlow model and track it with MLflow

The following cell trains a model using TensorFlow with CNNs layers and registers it with the MLflow Model Registry via the `mlflow.tensorflow.autolog()` function.

As above, we will try few different parameters and choose the best model.

In [0]:
params_list = [
        {'epochs': 3, 'convSize': 2},
        {'epochs': 8, 'convSize': 3}]
#iterate over few different tuning parameters
for params in params_list:
  TensorFlow_obj = TensorFlowModel(x_train, y_train, val_x.reshape(val_x.shape[0],-1), val_y, params, activation="softmax")
  print("Using paramerts={}".format(params))
  runID = TensorFlow_obj.mlflow_run()
  print("MLflow run_id={}".format(runID))

### 2.2 Explore the Model Registry API Workflow

When a new model version is created with the MLflow Model Registry's Python APIs, the name and version information is printed for future reference. You can also navigate to the MLflow Model Registry UI to view the new model version. 

Define the new model version as a variable in the cell below.

In [0]:
model_version = 4 # If necessary, replace this with the version corresponding to the new tensorflow model

### 2.3 Add a description to the new model version

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
client.update_model_version(
  name=model_name,
  version=model_version,
  description="This is the best model version. It's a TensorFlow model with CNNs layers"
)

### 2.4 Test the new model version in `Staging`

Before deploying a model to a production application, it is often best practice to test it in a staging environment. The following cells transition the new model version to `Staging` and evaluate its performance.

In [0]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage="Staging",
)

Evaluate the new model in `Staging`. A confusion matrix will be used again to evaluate.

In [0]:
model_staging_uri = "models:/{model_name}/staging".format(model_name=model_name)

model = mlflow.keras.load_model(model_staging_uri)
predictions = pd.DataFrame(model.predict(np.expand_dims(val_x.reshape(val_x.shape[0],-1),-1)))
PlotUtils.confusionMatrix(val_y, predictions.to_numpy().argmax(axis=1), classes= list(range(10)))

### 2.5 Deploy the new model version to `Production`

After verifying that the new model version performs well in staging, the following cells transition the model to `Production`, run new predictions and present the results using a confusion matrix.

**The MLflow Model Model Registry automatically uses the latest production version of the specified model, allowing you to update your production models without changing any application code**.

In [0]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage="Production"
)

In [0]:
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

model = mlflow.keras.load_model(model_production_uri)
predictions = pd.DataFrame(model.predict(np.expand_dims(val_x.reshape(val_x.shape[0],-1),-1)))
PlotUtils.confusionMatrix(val_y, predictions.to_numpy().argmax(axis=1), classes= list(range(10)))

## -- Delete models

When a model version is no longer being used, you can archive it or delete it. You can also delete an entire registered model; this removes all of its associated model versions.

### Workflow 1: Delete `Version 1` in the MLflow UI

To delete `Version 1` of the power forecasting model, open its corresponding Model Version page in the MLflow Model Registry UI. Then, select the drop-down arrow next to the version identifier and click `Delete`.

### Workflow 2: Delete `Version 1` using the MLflow API

The following cell permanently deletes `Version 1` of the power forecasting model. If you want to delete this model version, uncomment and execute the cell.

In [0]:
client.delete_model_version(name=model_name,version=1)

## Delete the Fashion MNIST model

If you want to delete an entire registered model, including all of its model versions, you can use the `MlflowClient.delete_registered_model()` to do so. This action cannot be undone.

**WARNING: The following cell permanently deletes the Fashion MNIST model, including all of its versions.** If you want to delete the model, uncomment and execute the cell.

In [0]:
#### REMOVE THIS CELL BEFORE DISTRIBUTING ####
client.delete_registered_model(name=model_name)